In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 29

## Confidence interval using the SE formula


In [ ]:
# get the data set of flight delays
united = Table.read_table('united_summer2015.csv')
united

In [ ]:
# Get a random sample size n = 100 of flight delays
sample_size = 100
my_sample = united.sample(sample_size, with_replacement = False).column("Delay")
my_sample

In [ ]:
# calculate the sample mean and sample SD
my_mean = np.mean(my_sample)
my_sd = np.std(my_sample)

In [ ]:
# calculate an approximate standard error (really we need to use a t-distribution)
approx_SE = my_sd/np.sqrt(sample_size)


In [ ]:
# calculate an approximate confidence interval using:  mean +/- 2 * SE
(my_mean - 2 * approx_SE, my_mean + 2 * approx_SE)


In [ ]:
# Did it capture the population mean mu? 
np.mean(united.column("Delay"))

## Confidence interval for a proportion

In [ ]:
# Let's look at the flights flying to JFK airport
united.group("Destination").where("Destination", "JFK")


In [ ]:
# calculate the population proportion (pi)
pop_proportion = np.mean(united.column('Destination') == "JFK")
pop_proportion

In [ ]:
sample_size = 900

proportions_900 = make_array()

for i in np.arange(10000):
    sampled_flights = united.sample(sample_size)
    sample_proportion = np.mean(sampled_flights.column('Destination') == "JFK")
    proportions_900 = np.append(proportions_900, sample_proportion)

In [ ]:
Table().with_column('Sample Proportions', proportions_900).hist()

In [ ]:
# calculate a confidence interval
# only valid when n * proportion and (n - 1) * proportion are greater than 10

sample_size = 900

one_sample = united.sample(sample_size)
sample_proportion = np.mean(one_sample.column('Destination') == "JFK")

SE_prop = .5/np.sqrt(sample_size)

(sample_proportion - 2 * SE_prop, sample_proportion + 2 * SE_prop)


In [ ]:
# does the confidence interval capture the population proportion of flights going to JFK? 
pop_proportion

## Prediction

Let's revisit Galton's predictions of children's heights based on their parent's heights...

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )
heights

In [ ]:
# look at a scatter plot of the relationship 
heights.scatter('MidParent')

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.25 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return close_points.column('Child').mean()   

In [ ]:
# predict the height for each child in the data set
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
    )

In [ ]:
# visualize the predicted heights
heights_with_predictions.scatter('MidParent')

## Association

Data on hybrid passenger cars sold in US 1997-2013:

- `vehicle`: model of the car
- `year`: year of manufacture
- `msrp`: manufacturer's suggested retail price in 2013 dollars
- `acceleration`: acceleration rate in km per hour per second
- `mpg`: fuel econonmy in miles per gallon
- `class`: the model's class.

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.sort('msrp', descending=True)

In [ ]:
# vehicles with higher mpg tend to cost less on average - surprising?
hybrid.scatter('mpg', 'msrp')

In [ ]:
# vehicles that accelerate faster tend to cost more, and have lower mpg (not as fuel efficient)
hybrid.scatter('acceleration', 'msrp')

### Correlation coefficient

In [ ]:
def r_scatter(r):
    plots.figure(figsize=(5,5))
    "Generate a scatter plot with a correlation approximately r"
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

In [ ]:
r_scatter(.3)

In [ ]:
r_scatter(0)

In [ ]:
r_scatter(-0.2)

In [ ]:
r_scatter(-.95)

In [ ]:
r_scatter(-.7)

### Calculating the correlation coefficient

To calculate the correlation coefficient r, we first convert our data to standardized units (by z-scoring out data)


In [ ]:
# Convert data to standard units
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x))/np.std(x)  

In [ ]:
hybrid

In [ ]:
hybrid2 = hybrid.select("msrp", "acceleration")

hybrid2 =  hybrid2.with_columns(
    'msrp (standard units)', standard_units(hybrid.column('msrp')),
    'acceleration (standard units)', standard_units(hybrid.column('acceleration')), 
)
hybrid2

In [ ]:
# Use of standard units does not change the point patterns
hybrid2.scatter('acceleration (standard units)', 'msrp (standard units)')

In [ ]:
# Use of standard units does not change the point patterns
hybrid2.scatter('acceleration', 'msrp')

In [ ]:
# we then calculate the product of the standardized units
hybrid2 = hybrid2.with_column(
    'product of standard units', hybrid2.column('acceleration (standard units)') * hybrid2.column('msrp (standard units)'))

hybrid2

In [ ]:
# r is the average of the products of standard units
r = np.sum(hybrid2.column('product of standard units'))/(hybrid2.num_rows -1)
r

In [ ]:
# we can create a function to calculate the correlation coefficient
def correlation(t, label_x, label_y):
    x_in_standard_units = standard_units(t.column(label_x))
    y_in_standard_units = standard_units(t.column(label_y))
    return np.sum(x_in_standard_units * y_in_standard_units)/(t.num_rows - 1)

In [ ]:
correlation(hybrid, 'acceleration', 'msrp')

In [ ]:
correlation(hybrid, 'mpg', 'msrp')

In [ ]:
#Order doesn't matter
correlation(hybrid, 'msrp', 'mpg')

## Correlation cautions

In [ ]:
# correlation only captures linear trends
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
# correlation is heavily influenced by outliers
anscombes = Table.read_table('anscombes.csv')
data1 = anscombes.where("dataset", "I")
data2 = anscombes.where("dataset", "II")
data3 = anscombes.where("dataset", "III")
data4 = anscombes.where("dataset", "IV")

In [ ]:
data1.scatter("x", "y")
plots.title("r = " + str(round(correlation(data1, "x", "y"), 3)));

In [ ]:
data2.scatter("x", "y")
plots.title("r = " + str(round(correlation(data2, "x", "y"), 3)));

In [ ]:
data3.scatter("x", "y")
plots.title("r = " + str(round(correlation(data3, "x", "y"), 3)));

In [ ]:
data4.scatter("x", "y")
plots.title("r = " + str(round(correlation(data4, "x", "y"), 3)));